<a href="https://colab.research.google.com/github/lmaheshds/S4---Assignment-QnA/blob/main/Copy_of_Copy_of_EVA4_Session_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 4, 3, padding=1) #input -? OUtput? RF
        self.bn1 = nn.BatchNorm2d(4)
        self.conv2 = nn.Conv2d(4, 8, 3, padding=1)
        self.bn2 = nn.BatchNorm2d(8)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv3 = nn.Conv2d(8, 12, 3, padding=1)
        self.bn3 = nn.BatchNorm2d(12)
        self.conv4 = nn.Conv2d(12, 16, 3, padding=1)
        self.bn4 = nn.BatchNorm2d(16)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.conv5 = nn.Conv2d(16, 24, 3)
        self.bn5 = nn.BatchNorm2d(24)
        self.conv6 = nn.Conv2d(24, 32, 3)
        self.bn6 = nn.BatchNorm2d(32)
        #self.conv7 = nn.Conv2d(32, 10, 3)
        self.fc1 = nn.Linear(32,10)
        self.drop_out = nn.Dropout(0.1)
        self.avg_pool2d = nn.AvgPool2d(3)

    def forward(self, x):
        x = self.bn1(F.relu(self.conv1(x)))
        x = self.bn2(F.relu(self.conv2(x)))
        x = self.pool1(self.bn3(F.relu(self.conv3(x))))
        x = self.bn4(F.relu(self.conv4(x)))
        x = self.pool2(self.bn5(F.relu(self.conv5(x))))
        x = self.bn6(F.relu(self.conv6(x)))
        x = self.avg_pool2d(x)
        x = x.view(-1,32)
        x = self.fc1(x)
        x = x.view(-1, 10)
        return F.log_softmax(x)

       # x = self.pool1(F.relu(self.conv2(F.relu(self.conv1(x)))))
        #x = self.pool2(F.relu(self.conv4(F.relu(self.conv3(x)))))
       # x = F.relu(self.conv6(F.relu(self.conv5(x))))
       # x = F.relu(self.conv7(x))
       # x = F.avg_pool2d(x,1)#x = F.avg_pool2d(x,(10,10)) # print(f.avg_pool2d(y, (1, 3), stride=1, padding=0, ceil_mode=False))
       # x = self.drop_out(x)
       # x = self.fc1(x)
       # x = x.view(-1, 10)
       # return F.log_softmax(x)

In [ ]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 4, 28, 28]              40
       BatchNorm2d-2            [-1, 4, 28, 28]               8
            Conv2d-3            [-1, 8, 28, 28]             296
       BatchNorm2d-4            [-1, 8, 28, 28]              16
            Conv2d-5           [-1, 12, 28, 28]             876
       BatchNorm2d-6           [-1, 12, 28, 28]              24
         MaxPool2d-7           [-1, 12, 14, 14]               0
            Conv2d-8           [-1, 16, 14, 14]           1,744
       BatchNorm2d-9           [-1, 16, 14, 14]              32
           Conv2d-10           [-1, 24, 12, 12]           3,480
      BatchNorm2d-11           [-1, 24, 12, 12]              48
        MaxPool2d-12             [-1, 24, 6, 6]               0
           Conv2d-13             [-1, 32, 4, 4]           6,944
      BatchNorm2d-14             [-1, 3

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:34: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [ ]:
torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [ ]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [ ]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 19):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0724, Accuracy: 9824/10000 (98%)



  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0426, Accuracy: 9884/10000 (99%)



  0%|          | 0/469 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0207, Accuracy: 9932/10000 (99%)



  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0192, Accuracy: 9941/10000 (99%)



loss=0.0027132052928209305 batch_id=468: 100%|██████████| 469/469 [00:12<00:00, 37.85it/s]



Test set: Average loss: 0.0208, Accuracy: 9937/10000 (99%)

